# Image classifier approach

This notebook describes my approach to image classification by some characteristics

## Features

### Color simmilarities

- Convert rgb values to light frequency
- plot values as curve to get color spectrum of the image
    - bin size for counting controls precision
- pick three highest peaks
- find image with the most similar peak values

### Feature similarities

- using ```cv.approxPolyDP()``` we find primitives in the image
    - [stackoverflow problem description](https://stackoverflow.com/questions/11424002/how-to-detect-simple-geometric-shapes-using-opencv)
- make list of primitives with type, average color, position and size
- compare with other images, turning on and off different similarity filters

## Similarity rating

Selecting the different aspects should give some normalized rating of how similar the images are.

## Addition
### Overall
I agree that adding different options that the user can turn on or off would make the system more flexible.

If we have time at the end, we could try building a GUI so the user can interact with these options.

It could also be useful to let the user adjust settings, like how many top colors to compare (e.g., top 1, 2, 3…), or choose between histogram-based or cluster-based approaches for color comparison.


### Some additions and thoughts:
- not sure if "light frequency" == HSV? 
- I would definetly use HSV
- we could cluster all HSV values using KMeans to find the main colors in the image. The resulting clusters would then contain both: color centers and their relative weight so that we can compare the overall color distribution in the images.



### To find similarities in where those colors are located in the image:
- we can add pixel coordinates as a feature in the KMeans clustering.
OR 
- a blurred version of the reference image and use it as a soft mask to compare how colors are spread out in the image. --> general idea of the color layout without focusing on exact color matches.
OR
- (I think you mentioned something like this before) split the image into a grid and calculate the main color for each section. -> low-resolution color map to compare with maps from other images.

In [1]:
# imports and path to images
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

In [2]:
N_CLUSTERS = 5
DIR = r"C:\Users\anton\OneDrive\Documents\HSD\sem4\DAISY_2025_images_for_bigdata"

### Loading Images with OpenCV and displaying them

In [3]:
def load_images_from_directory(directory):
    """
    Load all images from the specified directory using OpenCV.
    """
    images = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            img = cv2.imread(filepath)
            if img is not None:
                images.append((filename, img))
    return images

def display_image(images, img_id=None):
    """
    Display a list of images using Matplotlib.
    """
    if img_id is None:
        img_id = random.randint(0, len(images) - 1)
    else:
        img_id = img_id
    filename, img = images[img_id]
    plt.figure()
    plt.title(filename)
    # Convert BGR to RGB for displaying with Matplotlib
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    pass

def display_single_image(image):
    """
    Display a list of images using Matplotlib.
    """
    plt.figure()
    # Convert BGR to RGB for displaying with Matplotlib
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    pass

def downscale_image(image, factor=0.01):
    """
    Downscale images by a specified factor.
    """
    filename, img = image
    height, width = img.shape[:2]
    new_size = (int(width * factor), int(height * factor))
    downscaled_img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
    return (filename, downscaled_img)

def extract_dominant_colors(image, n_clusters=3):
    """
    Extract dominant colors from an image using KMeans clustering.
    Returns a n_clusters long list of rgb values 
    """
    # Unpack the tuple to get the filename and image
    # Convert the image to RGB (from BGR)
    # Reshape the image to a 2D array of pixels where each row is a pixel and each column is a color channel

    filename, img = image
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    pixels = img.reshape(-1, 3)
    
    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(pixels)
    
    # Get the cluster centers (dominant colors)
    dominant_colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_
    cluster_sizes = np.bincount(labels)

    # Get indices that would sort the cluster sizes in descending order
    sorted_indices = np.argsort(cluster_sizes)[::-1]

    # Sort both colors and cluster sizes using these indices
    dominant_colors = dominant_colors[sorted_indices]
    cluster_sizes = cluster_sizes[sorted_indices]
    
    # to percentages
    total_pixels = sum(cluster_sizes)
    percentages = (cluster_sizes / total_pixels) * 100

    return (filename, dominant_colors, percentages)

def plot_dominant_colors(all_colors, img_id=None):
    """
    Gets tuple with label and dominant colors and plots them
    """
    if img_id is None:
        img_id = random.randint(0, len(all_colors) - 1)
    else:
        img_id = img_id
        
    filename, colors, percentages = all_colors[img_id]
    
    # Create the plot
    plt.figure(figsize=(8, 2))
    plt.axis('off')
    plt.imshow([colors / 255])  # Normalize colors to [0, 1] for Matplotlib
    plt.title(filename)
    
    # Annotate with percentages
    for i, (color, percentage) in enumerate(zip(colors, percentages)):
        plt.text(
            x=i / len(colors) + 0.17,  # Position text slightly offset
            y=0.5,
            s=f"{percentage:.1f}%",
            color="white" if np.mean(color) < 128 else "black",  # Contrast text color
            fontsize=12,
            ha="center",
            va="center",
            transform=plt.gca().transAxes
        )
    
    plt.show()
    pass


### Clustering using KMeans

While clustering, I think it makes sense to choose attributes that have high value differences between images.

Because most images were taken using a smartphone, it makes little sense to choose `saturation` or `value` from `HSV`, because the automatic exposure and image post processing of smartphones ensures that these are similar across all images.


In [209]:
# Load and downscale images
images = load_images_from_directory(DIR)

# downscale images
downscaled_images = []
for img in images:
    downscaled_images.append(downscale_image(img))
# print(f"Downscaled {len(images)} images")

# extract dominant colors for all images
all_dominant_colors = []
for img in downscaled_images:
    all_dominant_colors.append(extract_dominant_colors(img, n_clusters=N_CLUSTERS))
# print(all_dominant_colors[testing_id][0])

# normalized
all_dominant_colors_normalized = []
for dominant_colors in all_dominant_colors:
    normalized_colors = dominant_colors[1] / 255
    norm_label = dominant_colors[0]
    percentages = dominant_colors[2]
    all_dominant_colors_normalized.append((norm_label, normalized_colors, percentages))
# print(all_dominant_colors_normalized[testing_id][0])

In [185]:
# Show progress of the workflow

# display_image(images)
# display_image(downscaled_images, img_id=testing_id)
# plot_dominant_colors(all_dominant_colors, img_id=testing_id)
# print(all_dominant_colors_normalized[testing_id][0])


# all_dominant_colors_normalized is a list of tuples
# tuple contains: n_cluster x rgb numpy array, img label.

# for i in range(len(all_dominant_colors)):
#     plot_dominant_colors(all_dominant_colors, img_id=i)

### Categorizing according to color similarity

To compare the `3 x n` arrays (where each row represents a color in normalized RGB format), we'll look into using some of the following methods:

#### Euclidean Distance:
- Treat each color as a point in 3D space (R, G, B).
- Calculate the Euclidean distance between the colors. Smaller distances indicate higher similarity.

$$d = \sqrt{(R_1 - R_2)^2 + (G_1 - G_2)^2 + (B_1 - B_2)^2}$$

Because we have multiple colors for each image we should compare the most dominant dominant of each respective image.


#### Cosine Similarity:
- Treat each color as a vector in 3D space.
- Compute the cosine of the angle between two vectors. A value closer to 1 indicates higher similarity.

#### Thresholding:
- Define a threshold for similarity (e.g., a maximum allowable distance).
- If the distance between two colors is below the threshold, consider them similar.

#### Other options:
##### Color Difference Metrics:
- Use perceptual color difference formulas like CIEDE2000 or CIE76, which account for how humans perceive color differences.

##### Cluster Matching:
- If comparing two sets of colors (e.g., two images), match the closest colors between the sets using a distance metric (e.g., Euclidean).
- Sum the distances for all matches to get an overall similarity score.

##### Weighted Comparison:
- Assign weights to the RGB channels based on their perceptual importance (e.g., green is more sensitive to the human eye).
- Use a weighted distance metric for comparison.




By combining these methods, you can create a robust similarity measure for your use case.

In [186]:
def euk_dis(img_1, img_2):
    """
    Calculate the Euclidean distance between the colors. Smaller distances indicate higher similarity.

    dist(x, y) = sqrt(dot(x, x) - 2 * dot(x, y) + dot(y, y))

    [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html)
    
    this function outputs the distance from every entry to every other entry.
    We're only interested in the diagonal of this matrix


    input: img_1 (numpy array), img_2 (numpy array)
    
    output: float for every distande
    
    output interpretation: `0`: Identical colors, √3: Maximum possible distance for 3 dim
    """
    all_distances_between_images = euclidean_distances(img_1, img_2)
    distances = np.diag(all_distances_between_images)
    return distances.T

def cos_dis(img_1, img_2):
    """
    cosine_distance = 1 - cosine_similarity

    K(X, Y) = <X, Y> / (||X||*||Y||)

    [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)
    
    this function outputs the cosine similarity from every entry to every other entry.
    We're only interested in the diagonal of this matrix

    input: img_1 (numpy array), img_2 (numpy array)
    
    output: float
    
    output interpretation: 0 = identical, 2 = maximally dissimilar.
    """
    all_similarities_between_images = cosine_similarity(img_1, img_2)
    similarity = np.diag(all_similarities_between_images)
    return [1, 1, 1] - similarity



# Example usage:
# test_wasserstein = wasserstein_similarity(all_dominant_colors_normalized[0][1],
#                                           all_dominant_colors_normalized[1][1])
# print(test_wasserstein)

# test = euk_dis(all_dominant_colors_normalized[0][1],
#         all_dominant_colors_normalized[1][1])
# print(cos_dis(all_dominant_colors_normalized[0][1],
#         all_dominant_colors_normalized[1][1]))
for image in all_dominant_colors_normalized:
    print(image)

('20250328_101537.jpg', array([[0.14509804, 0.13333333, 0.15294118],
       [0.48235294, 0.68627451, 0.66666667],
       [0.2745098 , 0.40784314, 0.43137255],
       [0.58823529, 0.2       , 0.14509804],
       [0.77647059, 0.31764706, 0.25490196]]), array([26.0230179 , 21.03580563, 19.56521739, 17.26342711, 16.11253197]))
('20250328_101618.jpg', array([[0.55686275, 0.57254902, 0.60784314],
       [0.50196078, 0.52941176, 0.56078431],
       [0.78823529, 0.56862745, 0.62745098],
       [0.41176471, 0.43137255, 0.45098039],
       [0.3254902 , 0.34117647, 0.35294118]]), array([48.91304348, 34.7826087 ,  8.69565217,  4.85933504,  2.74936061]))
('20250328_101653.jpg', array([[0.46666667, 0.51764706, 0.25490196],
       [0.35686275, 0.41176471, 0.18039216],
       [0.58823529, 0.58823529, 0.37647059],
       [0.21176471, 0.26666667, 0.09411765],
       [0.81960784, 0.72941176, 0.60784314]]), array([35.16624041, 26.85421995, 19.50127877, 11.89258312,  6.58567775]))
('20250328_101657.jpg', a

# Main comparison code-block
This code snippet compares the `euk_dis` and `cos_sim` and plots the `n` most similar images in the dataset.

hands over `k` most similar images to input image, based on euk_dis and cos_sim
returns labels of images that fit

## Similarity calculation approach

calculate the `euk_dis` or `cos_sim` for each cluster in the dominant_colors, making sure to compare the largest clusters, and moving on to the next smallest cluster.

To make sure that the the weight of each cluster distance is **not** equal we use a function to ensure proper similarity weight distribution.

The first implemenation will use a simple factor infront of the cluster similarity score:

`sim_score_0 * (1 - (0/n_clusters)` + `sim_score_1 * (1 - (1/n_clusters)` + `...` + `sim_score_n * (1 - (n/n_clusters)`

later on the `(1 - (0/n_clusters)` could be swapped for a more fitting weights function

In [187]:
# similarity(input_image, all_images):
"""
hands over 5 most similar images to input image, based on euk_dis and cos_sim

returns labels of images that fit
"""


def weights_function(similarity_scores, n_clusters=N_CLUSTERS):
    result = 0
    for i in range(len(similarity_scores)):
        result += similarity_scores[i] * (1 - (i/n_clusters))
    return result 

def compare_two_images(img_1, img_2):
    similarity_scores = wasserstein_distance(img_1, img_2)
    similarity = weights_function(similarity_scores)
    return similarity

def find_similarities(all_dominant_colors_normalized, img_idx_1):
    filename1, colors1, percentages1 = all_dominant_colors_normalized[img_idx_1]

    modified_list = []

    for idx, image_data in enumerate(all_dominant_colors_normalized):
        filename2, colors2, percentages2 = image_data
        
        if filename2 == filename1:
            # Add None similarity for the reference image
            modified_list.append((*image_data, None))
        else:
            # Calculate similarity and append with score
            similarity = compare_two_images(colors1, colors2)
            modified_list.append((*image_data, similarity))
    
    # Sort by similarity (position 3 in the tuple), exclude None values
    img_sorted_by_similarity = sorted(
        modified_list,
        key=lambda x: x[3] if x[3] is not None else float('inf')
    )
    
    # Return top 5 (excluding the reference image itself)
    return [item for item in img_sorted_by_similarity if item[3] is not None][:5]

def show_similar_images(images, list_of_images_to_show):
    """disclaimer: ai generated function!"""
    image_dict = {img[0]: img[1] for img in images}
    
    # Create a single figure with subplots
    fig, axes = plt.subplots(1, 5, figsize=(20, 5))  # 1 row, 5 columns
    
    # If there are fewer than 5 images, adjust the axes array
    if len(list_of_images_to_show) < 5:
        axes = axes.flat[:len(list_of_images_to_show)]
    
    for ax, show_data in zip(axes, list_of_images_to_show[:5]):  # Only show first 5
        filename = show_data[0]
        if filename in image_dict:
            _, _, percentages, similarity = show_data
            original_img = image_dict[filename]
            
            # Plot on the current axis
            ax.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
            ax.set_title(f"{filename}\nSimilarity: {similarity:.2f}")
            ax.axis('off')
    plt.tight_layout()
    plt.show()

# Future improvements:

n_clusters could be used to generate some similarity threshold

$\blacktriangleright$ **hypothesis:** image similarity might increase with `n_cluster`.

$\blacktriangleright$ $\blacktriangleright$ plot `n_clusters` against `similarity_score` to see when an increase in cluster size is no longer significant

In [191]:
from scipy.stats import wasserstein_distance

def wasserstein_similarity(img_1, img_2):
    """
    Calculate the Wasserstein distance (Earth Mover's Distance) between the colors of two images.
    
    This method compares the distributions of the dominant colors in the images.
    
    input: img_1 (numpy array), img_2 (numpy array)
    output: float (distance)
    
    output interpretation: Lower values indicate higher similarity.
    """
    # Flatten the RGB values into a single array for each image
    img_1_flat = img_1.flatten()
    img_2_flat = img_2.flatten()
    
    # Calculate the Wasserstein distance
    distance = wasserstein_distance(img_1_flat, img_2_flat)
    return distance

# Example usage:
# test_wasserstein = wasserstein_similarity(all_dominant_colors_normalized[0][1],
#                                           all_dominant_colors_normalized[1][1])
# print(test_wasserstein)

In [221]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.fft import fft2, fftshift

import cv2
import numpy as np
from scipy.signal import find_peaks

def detect_spectral_peaks(img, threshold=0.1):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    peaks = []
    for channel in cv2.split(lab):
        fft = np.fft.fft2(channel)
        mag = np.log1p(np.abs(np.fft.fftshift(fft)))
        coords = np.argwhere(mag > threshold * mag.max())
        peaks.extend(c[0], c[1], mag[c[0],c[1]]) # (y, x, magnitude)
    return sorted(peaks, key=lambda x: -x[2])[:100]  # Top 100 peaks

for i in range(10):
    detect_spectral_peaks(images[-1-i][1])

NameError: name 'c' is not defined

In [4]:
import os
import cv2
import numpy as np
from scipy.fft import fft2, fftshift
from skimage.measure import ransac
from skimage.transform import AffineTransform
from collections import defaultdict
from math import atan2, hypot, pi

# Your image loading functions
def load_images_from_directory(directory):
    """
    Load all images from the specified directory using OpenCV.
    """
    images = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            img = cv2.imread(filepath)
            if img is not None:
                images.append((filename, img))
    return images

def downscale_image(image, factor=0.25):
    """
    Downscale images by a specified factor.
    """
    filename, img = image
    height, width = img.shape[:2]
    new_size = (int(width * factor), int(height * factor))
    downscaled_img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
    return (filename, downscaled_img)

# Core fingerprinting functions
def detect_spectral_peaks(img, top_n=100, threshold=0.15):
    """Detect top spectral peaks in LAB color space"""
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    peaks = []
    
    for i, channel in enumerate(cv2.split(lab)):
        # Frequency analysis with windowing
        rows, cols = channel.shape
        win = np.outer(np.hamming(rows), np.hamming(cols))
        fft = fftshift(fft2(channel.astype(float) * win))
        mag = np.log1p(np.abs(fft))
        # Non-maximum suppression
        y, x = np.indices(mag.shape)
        peaks_ch = []
        for yc in range(1, rows-1):
            for xc in range(1, cols-1):
                if mag[yc, xc] > threshold * mag.max() and \
                   mag[yc, xc] == mag[yc-1:yc+2, xc-1:xc+2].max():
                    peaks_ch.append((yc, xc, mag[yc, xc]))
        
        # Keep top peaks per channel
        peaks_ch = sorted(peaks_ch, key=lambda x: -x[2])[:top_n//3]
        peaks.extend(peaks_ch)
    
    print("returning detected peaks in method")
    return sorted(peaks, key=lambda x: -x[2])[:top_n]

def form_anchor_pairs(peaks, max_dist=50):
    """Form spatial relationships between nearby peaks"""
    pairs = []
    for i in range(len(peaks)):
        for j in range(i+1, len(peaks)):
            y1, x1, m1 = peaks[i]
            y2, x2, m2 = peaks[j]
            
            dx = x2 - x1
            dy = y2 - y1
            if hypot(dx, dy) <= max_dist:
                pairs.append((
                    m1, m2,  # Magnitudes
                    dx, dy,   # Spatial offset
                    abs(m1 - m2)  # Magnitude difference
                ))
    print("returning formed anchor pairs")
    return pairs

def create_visual_hash(pair, angle_bins=8):
    """Create rotation-invariant hash from spatial relationship"""
    m1, m2, dx, dy, _ = pair
    r = hypot(dx, dy)
    theta = (atan2(dy, dx) + pi) % (2*pi)  # 0-2π range
    theta_q = int(theta / (2*pi/angle_bins))
    return (int(r), theta_q, int(m1*100), int(m2*100))

class VisualHashDB:
    """Database for storing and querying visual fingerprints"""
    def __init__(self):
        self.index = defaultdict(list)
        self.images = {}
    
    def add_image(self, img_id, image):
        # Convert to RGB and downscale
        rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        peaks = detect_spectral_peaks(rgb_img)
        pairs = form_anchor_pairs(peaks)
        
        for pair in pairs:
            h = create_visual_hash(pair)
            self.index[h].append({
                'img_id': img_id,
                'dx': pair[2],
                'dy': pair[3]
            })
        self.images[img_id] = rgb_img
    
    def query_image(self, query_img, min_matches=15):
        # Convert to RGB if needed
        if query_img.shape[2] == 3 and query_img.dtype == np.uint8:
            query_rgb = cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB)
        else:
            query_rgb = query_img
        
        print("detect spectral peaks")
        query_peaks = detect_spectral_peaks(query_rgb)
        print("form anchor pairs")
        query_pairs = form_anchor_pairs(query_peaks)
        
        candidates = defaultdict(list)
        for pair in query_pairs:
            print("create visual hash")
            h = create_visual_hash(pair)
            for entry in self.index.get(h, []):
                candidates[entry['img_id']].append((
                    (pair[2], pair[3]),  # Query offset
                    (entry['dx'], entry['dy'])  # DB offset
                ))
        
        # RANSAC verification
        for img_id, matches in candidates.items():
            if len(matches) >= min_matches:
                src = np.array([m[0] for m in matches])
                dst = np.array([m[1] for m in matches])
                
                model, inliers = ransac(
                    (src, dst),
                    AffineTransform,
                    min_samples=3,
                    residual_threshold=2.0,
                    max_trials=100
                )
                
                if np.sum(inliers) >= min_matches:
                    return img_id, np.sum(inliers)
        
        return None, 0

# Demo usage with your dataset
if __name__ == "__main__":
    # Configuration
    DIR = r"C:\Users\anton\OneDrive\Documents\HSD\sem4\poop_test"
    QUERY_DIR = os.path.join(DIR, "queries")
    DOWNSCALE_FACTOR = 0.25  # Adjust based on your needs
    
    # Initialize database
    db = VisualHashDB()
    
    # Load and process dataset
    print("Loading dataset images...")
    dataset_images = load_images_from_directory(DIR)
    print(f"Loaded {len(dataset_images)} images")
    
    print("Processing dataset...")
    for filename, img in dataset_images:
        # Downscale and add to DB
        _, downscaled = downscale_image((filename, img), DOWNSCALE_FACTOR)
        db.add_image(filename, downscaled)
    
    # Load query images
    print("\nLoading query images...")
    query_images = load_images_from_directory(QUERY_DIR)
    print(f"Loaded {len(query_images)} queries")
    
    # Process queries
    for filename, img in query_images:
        # Downscale query image
        _, downscaled = downscale_image((filename, img), DOWNSCALE_FACTOR)
        
        print(f"\nQuerying with {filename}:")
        matched_id, confidence = db.query_image(downscaled)
        
        if matched_id:
            print(f"Match found: {matched_id} (confidence: {confidence})")
            # For visual comparison (optional)
            cv2.imshow("Query", downscaled)
            cv2.imshow("Match", cv2.resize(db.images[matched_id], 
                                       (downscaled.shape[1], downscaled.shape[0])))
            cv2.waitKey(0)
        else:
            print("No match found")
    
    cv2.destroyAllWindows()

Loading dataset images...
Loaded 5 images
Processing dataset...


KeyboardInterrupt: 

# FFT approach

A different image classifier approach could be made, by referencing the process described in the [paper published by Shazam in 2000](https://www.ee.columbia.edu/~dpwe/papers/Wang03-shazam.pdf).

This paper describes a fourier transform to get an audio spectrum, we want to use HSV color space and create a color spectrum from the Hue by preformin an fft.

[Shazam hasing algorithm breakdown](https://www.youtube.com/watch?v=c1fJ0kKRNe4)


To find the closest hash in your dataset for a new image, you can combine **perceptual hashing algorithms** with **efficient similarity search techniques**. Below is a structured approach using Python libraries and existing algorithms:


#### **Algorithms & Methods**:
1. **Hamming Distance**  
   - Standard for comparing binary hashes (e.g., `hash1 - hash2`). Implemented via bitwise XOR and population count.  
   - Example: `scipy.spatial.distance.hamming` .
2. **k-d Trees**  
   - Optimized for low-dimensional spaces. Use `scipy.spatial.cKDTree` for Hamming/Cosine distances .
3. **Ball Trees**  
   - Better for high-dimensional data. Supported by `sklearn.neighbors.NearestNeighbors` .
4. **Locality-Sensitive Hashing (LSH)**  
   - Approximate method for large datasets. Libraries like `annoy` or `nmslib` can accelerate searches .
5. **Cosine Similarity**  
   - For vectorized hashes. Used in structural health monitoring (SHM) for multi-damage detection .

#### **Python Libraries**:
- **`scipy.spatial`** (cKDTree for nearest-neighbor search).
- **`sklearn.neighbors`** (NearestNeighbors with custom metrics).
- **`numpy`** (vectorized operations for batch comparisons).



# additional interesting features:

- ORB keypoints
- Texture similarity